<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [20]:
#!pip install polars

In [21]:
import polars as pl
import random
import string


# Generacion de Datos

In [22]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""ecyll""",13,30.150223,"2011-06-17 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4869
2,"""rwvfh""",10,56.175811,"2016-03-22 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2888
3,"""ayvup""",2,72.33966,"2017-12-06 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3209
4,"""vobvn""",11,62.712441,"2010-09-21 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3796
5,"""gfhvo""",13,48.958299,"2020-02-27 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4513


In [23]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [24]:

lazy_df = df.lazy()

df_resultado = (
    lazy_df
    .filter(pl.col("cantidad") < 5)  # 1. Filtrar ventas con cantidades menores a 5
    .with_columns((pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta"))  # 2. Agregar columna total_venta
    .drop("precio_unitario")  # 3. Eliminar columna precio_unitario
    .group_by("producto")  # 4. Agrupar por producto
    .agg([
        pl.sum("total_venta").alias("total_vendido"),  # Total vendido por producto
        pl.sum("cantidad").alias("cantidad_total_vendida")  # Cantidad total vendida por producto
    ])
    .sort("total_vendido", descending=True)  # 5. Ordenar por total vendido en orden descendente
)

# Ejecutar las operaciones y obtener el DataFrame en modo Lazy
resultado = df_resultado.collect()

# Visualizar el resultado
print(resultado)


shape: (198, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ wuanz    ┆ 392.641767    ┆ 4                      │
│ mrzus    ┆ 379.464266    ┆ 4                      │
│ ourah    ┆ 373.526537    ┆ 4                      │
│ pxade    ┆ 370.196011    ┆ 4                      │
│ jvvfr    ┆ 356.427658    ┆ 4                      │
│ …        ┆ …             ┆ …                      │
│ jlrla    ┆ 15.241239     ┆ 1                      │
│ inodj    ┆ 13.07095      ┆ 1                      │
│ bbgsp    ┆ 12.688378     ┆ 1                      │
│ wodmy    ┆ 12.625703     ┆ 1                      │
│ jtqpy    ┆ 11.891441     ┆ 1                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [25]:
def write_parquet_polars(dataframe):
    dataframe.write_parquet("resultados.parquet")

write_parquet_polars(resultado)

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [26]:
def read_parquet_polars(file_name):
    return pl.read_parquet(file_name)

df = read_parquet_polars('resultados.parquet')

In [27]:
df

producto,total_vendido,cantidad_total_vendida
str,f64,i64
"""wuanz""",392.641767,4
"""mrzus""",379.464266,4
"""ourah""",373.526537,4
"""pxade""",370.196011,4
"""jvvfr""",356.427658,4
…,…,…
"""jlrla""",15.241239,1
"""inodj""",13.07095,1
"""bbgsp""",12.688378,1
